<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

# Saving TF Models with SavedModel for TF Serving <a class="tocSkip">

In [1]:
import math
import os

import numpy as np
np.random.seed(123)
print("NumPy:{}".format(np.__version__))

import tensorflow as tf
tf.set_random_seed(123)
print("TensorFlow:{}".format(tf.__version__))

NumPy:1.14.5
TensorFlow:1.11.0


In [2]:
DATASETSLIB_HOME = '../datasetslib'
import sys
if not DATASETSLIB_HOME in sys.path:
    sys.path.append(DATASETSLIB_HOME)
%reload_ext autoreload
%autoreload 2
import datasetslib

from datasetslib import util as dsu
datasetslib.datasets_root = os.path.join(os.path.expanduser('~'),'datasets')
models_root = os.path.join(os.path.expanduser('~'),'models')

# Serving Model in TensorFlow

# Saving model with SavedModel

In [3]:
# Restart kernel to run the flag setting again
#tf.flags.DEFINE_integer('model_version', 1, 'version number of the model.')
model_name = 'mnist'
model_version = '1'
model_dir = os.path.join(models_root,model_name,model_version)

In [4]:
# get the MNIST Data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(os.path.join(datasetslib.datasets_root,'mnist'), one_hot=True)

x_train = mnist.train.images
x_test = mnist.test.images
y_train = mnist.train.labels
y_test = mnist.test.labels

# parameters
pixel_size = 28 
num_outputs = 10  # 0-9 digits
num_inputs = 784  # total pixels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/ubuntu/datasets/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/ubuntu/datasets/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /home/ubuntu/datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/ubuntu/datasets/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
def mlp(x, num_inputs, num_outputs,num_layers,num_neurons):
    w=[]
    b=[]
    for i in range(num_layers):
        # weights
        w.append(tf.Variable(tf.random_normal( \
                              [num_inputs if i==0 else num_neurons[i-1], \
                               num_neurons[i]]), \
                             name="w_{0:04d}".format(i) \
                            ) \
                ) 
        # biases
        b.append(tf.Variable(tf.random_normal( \
                              [num_neurons[i]]), \
                             name="b_{0:04d}".format(i) \
                            ) \
                )                   
    w.append(tf.Variable(tf.random_normal(
                          [num_neurons[num_layers-1] if num_layers > 0 else num_inputs,
                           num_outputs]),name="w_out"))
    b.append(tf.Variable(tf.random_normal([num_outputs]),name="b_out"))
    
    # x is input layer
    layer = x
    # add hidden layers
    for i in range(num_layers):
        layer = tf.nn.relu(tf.matmul(layer, w[i]) + b[i])
    # add output layer
    layer = tf.matmul(layer, w[num_layers]) + b[num_layers]
    model = layer
    probs = tf.nn.softmax(model)
    
    return model,probs

In [6]:
tf.reset_default_graph()

In [7]:
# input images
serialized_tf_example = tf.placeholder(tf.string, name='tf_example')
feature_configs = {'x': tf.FixedLenFeature(shape=[784], dtype=tf.float32),}
tf_example = tf.parse_example(serialized_tf_example, feature_configs)
x_p = tf.identity(tf_example['x'], name='x_p')  # use tf.identity() to assign name

# target output
y_p = tf.placeholder(dtype=tf.float32, name="y_p", shape=[None, num_outputs]) 

In [8]:
num_layers = 2
num_neurons = []
for i in range(num_layers):
    num_neurons.append(256)
    
learning_rate = 0.01
n_epochs = 50
batch_size = 100
n_batches = mnist.train.num_examples//batch_size

model,probs = mlp(x=x_p, 
            num_inputs=num_inputs, 
            num_outputs=num_outputs, 
            num_layers=num_layers, 
            num_neurons=num_neurons)

# loss function
#loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(model), axis=1))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y_p))
# optimizer function
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

train_op = optimizer.minimize(loss)

predictions_check = tf.equal(tf.argmax(probs,1), tf.argmax(y_p,1))
accuracy_function = tf.reduce_mean(tf.cast(predictions_check, tf.float32))

values, indices = tf.nn.top_k(probs, 10)
table = tf.contrib.lookup.index_to_string_table_from_tensor(
  tf.constant([str(i) for i in range(10)]))
prediction_classes = table.lookup(tf.to_int64(indices))

with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        epoch_loss = 0.0
        for batch in range(n_batches):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            _,batch_loss = tfs.run([train_op,loss], feed_dict={x_p: x_batch, y_p: y_batch})
            epoch_loss += batch_loss 
        average_loss = epoch_loss / n_batches
        print("epoch: {0:04d}   loss = {1:0.6f}".format(epoch,average_loss))
    accuracy_score = tfs.run(accuracy_function, feed_dict={x_p: x_test, y_p: y_test })
    print("accuracy={0:.8f}".format(accuracy_score))

    # save the model
    
    # definitions for saving the models
    builder = tf.saved_model.builder.SavedModelBuilder(model_dir)

    # build signature_def_map

    classification_inputs = tf.saved_model.utils.build_tensor_info(
      serialized_tf_example)
    classification_outputs_classes = tf.saved_model.utils.build_tensor_info(
      prediction_classes)
    classification_outputs_scores = tf.saved_model.utils.build_tensor_info(values)

    classification_signature = (
      tf.saved_model.signature_def_utils.build_signature_def(
          inputs={
              tf.saved_model.signature_constants.CLASSIFY_INPUTS:
                  classification_inputs
          },
          outputs={
              tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES:
                  classification_outputs_classes,
              tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES:
                  classification_outputs_scores
          },
          method_name=tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME))

    tensor_info_x = tf.saved_model.utils.build_tensor_info(x_p)
    tensor_info_y = tf.saved_model.utils.build_tensor_info(probs)

    prediction_signature = (
          tf.saved_model.signature_def_utils.build_signature_def(
              inputs={'inputs': tensor_info_x},
              outputs={'outputs': tensor_info_y},
              method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

    legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
    builder.add_meta_graph_and_variables(
      tfs, [tf.saved_model.tag_constants.SERVING],
      signature_def_map={
          'predict_images':
              prediction_signature,
          tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
              classification_signature,
      },
      legacy_init_op=legacy_init_op)
    
    builder.save()

print('Run following command:')
print('tensorflow_model_server --model_name=mnist --model_base_path={}'
      .format(os.path.join(models_root,model_name)))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

epoch: 0000   loss = 59.730989
epoch: 0001   loss = 14.145552
epoch: 0002   loss = 8.525061
epoch: 0003   loss = 5.947919
epoch: 0004   loss = 4.390820
epoch: 0005   loss = 3.441660
epoch: 0006   loss = 2.671090
epoch: 0007   loss = 2.208723
epoch: 0008   loss = 1.769823
epoch: 0009   loss = 1.467833
epoch: 0010   loss = 1.221096
epoch: 0011   loss = 1.060124
epoch: 0012   loss = 0.887587
epoch: 0013   loss = 0.750789
epoch: 0014   loss = 0.643013
epoch: 0015   loss = 0.539258
epoch: 0016   loss = 0.469765
epoch: 0017   loss = 0.399119
epoch: 0018   loss = 0.329758
epoch: 0019   loss = 0.281580
epoch: 0020   loss = 0.240632
epoch: 0021   loss = 0.204305
epoch: 0022   loss = 0.170387
epoch: 0023   loss = 0.149727
epoch: 0024   loss = 0.125530
epoch: 0025   loss = 0.104190
epoch: 0026   loss = 0.0899